# Evaluate Predictive Power of ECG Features (Baseline)

In [48]:
import pandas as pd
from ptbxlae.dataprocessing.dataModules import PtbxlCleanDM
from tqdm.auto import tqdm
import os
import numpy as np
import neurokit2 as nk
import torch

In [49]:
ecg_features = pd.read_csv("../data/unig_features.csv", index_col='ecg_id')

for c in ecg_features.columns:
    ecg_features[c] = ecg_features[c].fillna(ecg_features[c].mean())
    
ecg_features

,P_On_I,P_On_II,P_On_III,P_On_aVR,P_On_aVL,P_On_aVF,P_On_V1,P_On_V2,P_On_V3,P_On_V4,P_On_V5,P_On_V6,P_DurFull_I,P_DurFull_II,P_DurFull_III,P_DurFull_aVR,P_DurFull_aVL,P_DurFull_aVF,P_DurFull_V1,P_DurFull_V2,P_DurFull_V3,P_DurFull_V4,P_DurFull_V5,P_DurFull_V6,QRS_On_I,QRS_On_II,QRS_On_III,QRS_On_aVR,QRS_On_aVL,QRS_On_aVF,QRS_On_V1,QRS_On_V2,QRS_On_V3,QRS_On_V4,QRS_On_V5,QRS_On_V6,QRS_Dur_I,QRS_Dur_II,QRS_Dur_III,QRS_Dur_aVR,...,T-_Dur_V6,P-_Dur_V6,P_Amp_V6,P_Dur_V6,Pp_Amp_V6,T_Amp_V6,Pp_Dur_V6,Tp_Amp_V6,T_Dur_V6,Tp_Dur_V6,T-_Dur_aVR,P-_Dur_aVR,P_Amp_aVR,P_Dur_aVR,Pp_Amp_aVR,T_Amp_aVR,Pp_Dur_aVR,Tp_Amp_aVR,T_Dur_aVR,Tp_Dur_aVR,T-_Dur_aVL,P-_Dur_aVL,P_Amp_aVL,P_Dur_aVL,Pp_Amp_aVL,T_Amp_aVL,Pp_Dur_aVL,Tp_Amp_aVL,T_Dur_aVL,Tp_Dur_aVL,T-_Dur_aVF,P-_Dur_aVF,P_Amp_aVF,P_Dur_aVF,Pp_Amp_aVF,T_Amp_aVF,Pp_Dur_aVF,Tp_Amp_aVF,T_Dur_aVF,Tp_Dur_aVF
ecg_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
13481,240.0,240.0,240.0,240.0,240.0,240.0,240.000000,240.000000,240.000000,240.000000,240.000000,240.00000,112.0,112.0,112.0,112.0,112.0,112.0,112.000000,112.000000,112.000000,112.000000,112.000000,112.000000,442.0,436.0,444.0,438.0,444.0,448.0,444.0000,436.000000,436.000000,436.000000,436.00000,440.000000,72.0,80.0,68.0,78.0,...,0.000000,0.000000,0.090000,112.000000,0.000000,0.131000,0.000000,0.000000,152.000000,0.000000,156.0,112.0,-0.133,112.0,0.000,-0.154,0.0,0.000,156.0,0.0,0.0,0.0,0.040,112.0,0.000,0.041,0.0,0.0,154.0,0.0,0.0,0.0,0.107,112.0,0.00,0.122,0.0,0.000,162.0,0.0
13522,294.0,294.0,294.0,294.0,294.0,294.0,294.000000,294.000000,294.000000,294.000000,294.000000,294.00000,108.0,108.0,108.0,108.0,108.0,108.0,108.000000,108.000000,108.000000,108.000000,108.000000,108.000000,424.0,436.0,424.0,430.0,424.0,426.0,422.0000,422.000000,426.000000,428.000000,436.00000,438.000000,60.0,74.0,90.0,70.0,...,0.000000,0.000000,0.056000,108.000000,0.000000,0.113000,0.000000,0.000000,146.000000,0.000000,148.0,108.0,-0.133,108.0,0.000,-0.198,0.0,0.000,148.0,0.0,0.0,0.0,0.046,108.0,0.000,0.104,0.0,0.0,148.0,0.0,0.0,0.0,0.101,108.0,0.00,0.110,0.0,0.000,134.0,0.0
4811,284.0,284.0,284.0,284.0,284.0,284.0,284.000000,284.000000,284.000000,284.000000,284.000000,284.00000,122.0,122.0,122.0,122.0,122.0,122.0,122.000000,122.000000,122.000000,122.000000,122.000000,122.000000,426.0,424.0,422.0,430.0,424.0,422.0,432.0000,436.000000,436.000000,428.000000,424.00000,426.000000,70.0,90.0,92.0,54.0,...,0.000000,0.000000,0.085000,122.000000,0.000000,0.167000,0.000000,0.000000,208.000000,0.000000,224.0,122.0,-0.111,122.0,0.000,-0.174,0.0,0.000,224.0,0.0,0.0,54.0,-0.041,54.0,0.038,0.069,68.0,0.0,108.0,0.0,0.0,0.0,0.113,122.0,0.00,0.111,0.0,0.000,162.0,0.0
8281,268.0,268.0,268.0,268.0,268.0,268.0,268.000000,268.000000,268.000000,268.000000,268.000000,268.00000,108.0,108.0,108.0,108.0,108.0,108.0,108.000000,108.000000,108.000000,108.000000,108.000000,108.000000,436.0,436.0,450.0,436.0,440.0,440.0,444.0000,434.000000,432.000000,432.000000,436.00000,438.000000,88.0,90.0,72.0,90.0,...,0.000000,0.000000,0.049000,108.000000,0.000000,0.234000,0.000000,0.000000,188.000000,0.000000,204.0,108.0,-0.097,108.0,0.000,-0.219,0.0,0.000,204.0,0.0,0.0,36.0,-0.010,36.0,0.036,0.048,72.0,0.0,162.0,0.0,0.0,0.0,0.066,108.0,0.00,0.181,0.0,0.000,204.0,0.0
5746,324.0,324.0,324.0,324.0,324.0,324.0,324.000000,324.000000,324.000000,324.000000,324.000000,324.00000,62.0,62.0,62.0,62.0,62.0,62.0,62.000000,62.000000,62.000000,62.000000,62.000000,62.000000,462.0,452.0,454.0,464.0,460.0,452.0,448.0000,448.000000,450.000000,452.000000,458.00000,452.000000,88.0,80.0,100.0,80.0,...,0.000000,0.000000,0.065000,62.000000,0.000000,0.737000,0.000000,0.000000,216.000000,0.000000,208.0,62.0,-0.102,62.0,0.000,-0.497,0.0,0.000,208.0,0.0,0.0,44.0,-0.031,44.0,0.016,0.268,18.0,0.0,214.0,0.0,0.0,0.0,0.115,62.0,0.00,0.231,0.0,0.000,208.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,..

In [50]:
dm = PtbxlCleanDM(root_folder="../data")
dm.setup(stage='test')

test_metadata = dm.test_ds.dataset.metadata.iloc[dm.test_ds.indices]

combined_df = pd.merge(test_metadata.set_index('ecg_id'), ecg_features, how='left', left_index=True, right_index=True)
combined_df

,patient_id,patient_id.1,age,sex,height,weight,nurse,site,device,recording_date,report,scp_codes,heart_axis,infarction_stadium1,infarction_stadium2,validated_by,second_opinion,initial_autogenerated_report,validated_by_human,baseline_drift,static_noise,burst_noise,electrodes_problems,extra_beats,pacemaker,strat_fold,filename_lr,filename_hr,P_On_I,P_On_II,P_On_III,P_On_aVR,P_On_aVL,P_On_aVF,P_On_V1,P_On_V2,P_On_V3,P_On_V4,P_On_V5,P_On_V6,...,T-_Dur_V6,P-_Dur_V6,P_Amp_V6,P_Dur_V6,Pp_Amp_V6,T_Amp_V6,Pp_Dur_V6,Tp_Amp_V6,T_Dur_V6,Tp_Dur_V6,T-_Dur_aVR,P-_Dur_aVR,P_Amp_aVR,P_Dur_aVR,Pp_Amp_aVR,T_Amp_aVR,Pp_Dur_aVR,Tp_Amp_aVR,T_Dur_aVR,Tp_Dur_aVR,T-_Dur_aVL,P-_Dur_aVL,P_Amp_aVL,P_Dur_aVL,Pp_Amp_aVL,T_Amp_aVL,Pp_Dur_aVL,Tp_Amp_aVL,T_Dur_aVL,Tp_Dur_aVL,T-_Dur_aVF,P-_Dur_aVF,P_Amp_aVF,P_Dur_aVF,Pp_Amp_aVF,T_Amp_aVF,Pp_Dur_aVF,Tp_Amp_aVF,T_Dur_aVF,Tp_Dur_aVF
ecg_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
21416,15000,15000.0,67.0,0.0,NaN,NaN,0.0,0.0,CS100 3,2000-10-03 08:33:04,sinusrhythmus verdacht auf p-sinistrocardiale ...,"{'ILMI': 100.0, 'IRBBB': 100.0, 'ABQRS': 0.0, ...",ALAD,Stadium II-III,NaN,NaN,False,True,False,NaN,NaN,NaN,NaN,NaN,NaN,1.0,records100/21000/21416_lr,records500/21000/21416_hr,230.0,230.0,230.0,230.0,230.0,230.0,230.0,230.0,230.0,230.0,230.0,230.0,...,0.0,0.0,0.075,124.0,0.0,0.030,0.0,0.0,92.0,0.0,158.0,124.0,-0.114,124.0,0.0,-0.079,0.0,0.0,158.0,0.0,128.0,124.0,-0.047,124.0,0.000,-0.061,0.0,0.000,128.0,0.0,0.0,0.0,0.142,124.0,0.000,0.141,0.0,0.000,146.0,0.0
9809,18061,18061.0,30.0,1.0,NaN,60.0,0.0,0.0,CS-12 E,1993-02-14 12:41:46,sinusrhythmus normales ekg,"{'NORM': 100.0, 'SR': 0.0}",NaN,NaN,NaN,NaN,False,False,True,NaN,NaN,NaN,NaN,NaN,NaN,2.0,records100/09000/09809_lr,records500/09000/09809_hr,352.0,352.0,352.0,352.0,352.0,352.0,352.0,352.0,352.0,352.0,352.0,352.0,...,0.0,0.0,0.092,76.0,0.0,0.159,0.0,0.0,154.0,0.0,174.0,76.0,-0.159,76.0,0.0,-0.196,0.0,0.0,174.0,0.0,0.0,76.0,-0.049,76.0,0.000,0.056,0.0,0.000,164.0,0.0,0.0,0.0,0.188,76.0,0.000,0.141,0.0,0.000,174.0,0.0
1228,3612,3612.0,78.0,0.0,170.0,63.0,10.0,1.0,AT-6 C 5.5,1987-10-02 11:38:52,frequent premature atrial contraction(s). poss...,"{'RAO/RAE': 100.0, 'PRC(S)': 0.0, 'SR': 0.0}",NaN,NaN,NaN,0.0,True,False,True,NaN,NaN,NaN,NaN,NaN,NaN,5.0,records100/01000/01228_lr,records500/01000/01228_hr,262.0,262.0,262.0,262.0,262.0,262.0,262.0,262.0,262.0,262.0,262.0,262.0,...,0.0,0.0,0.062,90.0,0.0,0.100,0.0,0.0,186.0,0.0,214.0,90.0,-0.119,90.0,0.0,-0.129,0.0,0.0,214.0,0.0,174.0,90.0,-0.028,90.0,0.000,-0.022,0.0,0.000,174.0,0.0,0.0,0.0,0.140,90.0,0.000,0.146,0.0,0.000,220.0,0.0
2683,4034,4034.0,71.0,0.0,168.0,65.0,10.0,1.0,AT-6 6,1989-04-13 14:50:48,sinus rhythm. rightward axis. possible left po...,"{'IRBBB': 100.0, 'LPFB': 100.0, 'SR': 0.0}",NaN,NaN,NaN,0.0,False,False,True,NaN,NaN,NaN,NaN,NaN,NaN,6.0,records100/02000/02683_lr,records500/02000/02683_hr,238.0,238.0,238.0,238.0,238.0,238.0,238.0,238.0,238.0,238.0,238.0,238.0,...,0.0,0.0,0.046,120.0,0.0,0.365,0.0,0.0,198.0,0.0,208.0,120.0,-0.061,120.0,0.0,-0.243,0.0,0.0,208.0,0.0,92.0,56.0,-0.008,56.0,0.027,0.047,64.0,-0.012,42.0,92.0,0.0,0.0,0.064,120.0,0.000,0.242,0.0,0.000,178.0,0.0
15928,5644,5644.0,69.0,0.0,165.0,NaN,1.0,1.0,AT-6 C 5.5,1996-04-12 11:33:00,premature atrial contraction(s). sinus rhythm....,"{'NORM': 80.0, 'PAC': 0.0, 'SR': 0.0}",NaN,NaN,NaN,0.0,False,False,True,NaN,NaN,NaN,NaN,NaN,NaN,9.0,records100/15000/15928_lr,records500/15000/15928_hr,274.0,274.0,274.0,274.0,274.0,274.0,274.0,274.0,274.0,274.0,274.0,274.0,...,0.0,0.0,0.055,96.0,0.0,0.155,0.0,0.0,268.0,0.0,262.0,96.0,-0.103,96.0,0.0,-0.171,0.0,0.0,262.0,0.0,0.0,0.0,0.045,96.0,0.000,0.108,0.0,0.000,238.0,0.0,0.0,0.0,0.071,96.0,0.000,0.072,0.0,0.000,258.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

In [51]:
import ast
from util import eval_predictive_power_binary_outcome

all_scps = pd.read_csv("../data/scp_statements.csv", index_col=0)

def ptb_val_to_pseudobinary_label(scp_code_of_interest: str, this_recording_scp_codes: str):
    if scp_code_of_interest not in this_recording_scp_codes.keys():
        return 0.0
    elif scp_code_of_interest in this_recording_scp_codes.keys() and this_recording_scp_codes[scp_code_of_interest] == 100.0:
        return 1.0
    else:
        return float('nan')


results = list()
for scp_code in tqdm(all_scps.index.to_list()):
    combined_df[f'scp.{scp_code}'] = combined_df['scp_codes'].apply(lambda codes: ptb_val_to_pseudobinary_label(scp_code, ast.literal_eval(codes)))
    relevant_df = combined_df[~combined_df[f'scp.{scp_code}'].isna()]

    res = eval_predictive_power_binary_outcome(relevant_df[ecg_features.columns], relevant_df[f'scp.{scp_code}'])
    res['Target'] = all_scps.loc[scp_code]['description']
    results.append(res)


results_df = pd.DataFrame.from_records(results)
results_df.nlargest(n=50, columns=['Avg CV score'])


  0%|          | 0/71 [00:00<?, ?it/s]

/tmp/ipykernel_3446219/2564647245.py:17: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  combined_df[f'scp.{scp_code}'] = combined_df['scp_codes'].apply(lambda codes: ptb_val_to_pseudobinary_label(scp_code, ast.literal_eval(codes)))
/home/isears/VirtualEnvironments/default/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_res

,Total usable,% positive,Avg CV score,Target
15,1505,0.021927,0.994915,complete left bundle branch block
14,1505,0.021927,0.959890,complete right bundle branch block
8,1502,0.071238,0.941613,left anterior fascicular block
24,1505,0.007973,0.936700,left posterior fascicular block
4,1333,0.399850,0.918334,normal ECG
6,1451,0.062715,0.875400,anteroseptal myocardial infarction
10,1501,0.053298,0.873282,incomplete right bundle branch block
9,1491,0.057009,0.871677,non-specific ischemic
26,1504,0.009973,0.843511,subendocardial injury in anterolateral leads
27,1505,0.006645,0.829431,ischemic in lateral leads
